# Extreme Learning Machine (ELM)
### Neural network tanpa optimasi iteratif

Pada dasarnya, ELM adalah *feed forward neural network* (NN). Berikut ini adalah bentuk paling sederhana dari NN dengan suatu fungsi aktivasi nonlinear, $\sigma(\cdot)$:
$$\widehat{\mathbf{Y}}=\mathbf{W}_2\sigma(\mathbf{W}_1\mathbf{X})$$

Di sini, kita membuat prediksi ($\widehat{\mathbf{Y}}\in\mathbb{R}^{m \times c}$) bermodal suatu input ($\mathbf{X}\in\mathbb{R}^{m \times n}$), di mana $\mathbf{W}_1\in\mathbb{R}^{n \times h}$ adalah matriks bobot input (untuk mengkoneksikan input ke *hidden* layer), serta $\mathbf{W}_2\in\mathbb{R}^{h \times c}$ adalah matriks bobot output (untuk mengkoneksikan *hidden* layer (terakhir) ke output). Yang menarik (dan membedakannya dengan NN "pada umumnya") adalah:
- Matriks bobot input ke *hidden* layer (serta bobot hidden layer ke hidden layer selanjutnya, jika ada beberapa layer) tidak perlu dioptimasi. Dengan kata lain, cukup diacak saja. Sekilas nampak aneh, namun percayalah, ini berfungsi dengan cukup baik :p
- Optimasi tidak dilakukan secara iteratif seperti gradient descent dan sejenisnya, namun cukup dengan satu langkah saja: dengan **pseudoinverse**

#### Klasifikasi dataset *fisher iris* dengan ELM

![Iris_versicolor](https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/Iris_versicolor_3.jpg/440px-Iris_versicolor_3.jpg)

Sebagai contoh, kita akan menggunakan dataset bunga iris. Dataset ini memiliki 4 *feature* (panjang sepal, lebar sepal, panjang daun bunga, dan lebar daun bunga) dan 3 kelas (*Iris setosa, Iris virginica,* dan *Iris versicolor*). Dataset cukup dibagi menjadi 2: training set (80% dari total data) dan testing set (20% dari total data).

In [324]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# demi kesamaan hasil
# 123 dipilih berdasarkan mood penulis
np.random.seed(123)

X, y                    = load_iris(return_X_y=True)
X_tr, X_te, y_tr, y_te  = train_test_split(X, y, test_size=0.2)

# Konon, neural net akan bekerja lebih baik dengan
# data yang dinormalisasi
scaler = StandardScaler().fit(X_tr)
X_tr   = scaler.transform(X_tr)
X_te   = scaler.transform(X_te)

# jumlah baris dan kolom (feature) input
m, n                    = X_tr.shape

# sebagai contoh, jumlah neuron dalam hidden layer adalah 10
h                       = 10

# dictionary untuk mengkonversi kode numerik kelas menjadi string
name_by_code            = {
                            0: 'Iris setosa',
                            1: 'Iris virginica',
                            2: 'Iris versicolor',
                          }

Biasanya label *ground truth* di-*encode* terlebih dahulu dalam format one-hot encoding. Misal:
- Kelas 0 (Iris setosa) di-*encode* menjadi \[0,0,1\]
- Kelas 1 (Iris virginica) di-*encode* menjadi \[0,1,0\]
- Kelas 2 (Iris versicolor) di-*encode* menjadi \[1,0,0\]

Tujuan *encode* adalah agar hasil prediksi bisa diterjemahkan sebagai nilai *confidence* atau tingkat keyakinan (penejelasan *confidence* di bagian akhir). Sklearn sudah menyediakan fungsi one-hot encoding dengan menggunakan kelas LabelBinarizer:

In [325]:
from sklearn.preprocessing import LabelBinarizer
y_tr_bin = LabelBinarizer().fit_transform(y_tr)
y_te_bin = LabelBinarizer().fit_transform(y_te)

# jumlah kolom matriks output
h        = y_tr_bin.shape[1]

print("Contoh label asli:",y_tr[:3])
print("Contoh label ter-encode:",y_tr_bin[:3].tolist())

# demi konsistensi dengan formula matematika, dataset akan direpresentasikan sebagai column-vector.
X_tr, X_te, y_tr_bin, y_te_bin = X_tr.T, X_te.T, y_tr_bin.T, y_te_bin.T

Contoh label asli: [2 2 0]
Contoh label ter-encode: [[0, 0, 1], [0, 0, 1], [1, 0, 0]]


Setelah dataset siap, langkah pertama adalah menyiapkan matriks bobot input. Seperti yang sudah disebutkan, matriks bobot input cukup diacak. Sebagai contoh, kita menggunakan nilai acak dengan distribusi standard normal, $\mathbf{W}_1\sim\mathcal{N}(0, 1)$.

In [326]:
# W1 berukuran h x n. Artinya, dia akan memetakan input X yang berdimensi n menjadi
# hidden representation H berdimensi h
W1 = np.random.randn(h, n)

Langkah selanjutnya adalah estimasi $\mathbf{W}_2$. Pada tahap ini, kita sedang melakukan training. Misal, label ter-*encode* pada training dataset direpresentasikan sebagai $\mathbf{Y}$. Substitusikan dalam persamaan, kita akan memperoleh:
$$\mathbf{Y}=\mathbf{W}_2\sigma(\mathbf{W}_1\mathbf{X})$$
Maka, $\mathbf{W}_2$ optimal dapat diperoleh sebagai berikut:
$$\widehat{\mathbf{W}}_2=\mathbf{Y}\sigma(\mathbf{W}_1\mathbf{X})^+$$
*Superscript* "+" menandakan operasi (Moore-Penrose) pseudoinverse. Pseudoinverse suatu matriks bisa diperoleh dengan prosedur yang melibatkan *singular value decomposition* (SVD). Namun, numpy sudah menyediakan fungsi pinv() untuk menghitung pseudoinverse.

Untuk fungsi aktivasi, pada contoh ini kita akan menggunakan fungsi sigmoid:

$$\sigma(x)=\frac{1}{1+e^{-x}}$$

In [327]:
# fungsi sigmoid
def sig(x):
    return 1 / (1 + np.exp(-x))

# estimasi (a.k.a. training) W2 dengan pseudoinverse
W2 = y_tr_bin @ np.linalg.pinv(sig(W1 @ X_tr))

Sampai di sini kita sudah bisa melakukan prediksi (*inference*):
$$\widehat{\mathbf{Y}}=\widehat{\mathbf{W}}_2\sigma(\mathbf{W}_1\mathbf{X})$$

Sebagai contoh, kita mempunyai data bunga iris baru sbb:
- panjang sepal = 5.2 cm
- lebar sepal = 3.5 cm
- panjang daun bunga = 4 cm
- lebar daun bunga = 2 cm

In [328]:
X_new = np.array([[5.2, 3.5, 4.0, 2.0]]).T
y_hat = (W2 @ sig(W1 @ X_new))

label = y_hat.argmax()

print('prediksi kelas: %d (%s)' % (label, name_by_code[label]))

prediksi kelas: 2 (Iris versicolor)


Benarkah? Entahlah. Itu hanya data contoh. Untuk mengevaluasi performa (akurasi) model, kita bisa menggunakan testing set:

In [329]:
pred = (W2@sig(W1 @ X_te)).argmax(axis=0)
acc  = np.mean([pred == y_te])
print('Akurasi ELM: %.2f%%' % (acc*100))

Akurasi ELM: 93.33%


Mari kita coba bandingkan dengan neural network bawaan sklearn dengan parameter default.

In [330]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier().fit(X_tr.T, y_tr)
acc = np.mean([clf.predict(X_te.T) == y_te])
print('Akurasi neural net (sklearn): %.2f%%' % (acc*100))

Akurasi neural net (sklearn): 93.33%


/home/ghora/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Ternyata tidak jauh berbeda. Cukup bagus lah, terlebih model kita sangat sederhana. Namun ada poin yang menjadi kelebihan ELM: Tidak membutuhkan update matrix bobot secara iteratif. Artinya? Kita tidak perlu bingung memilih berapa iterasi hingga konvergen. Kita juga tidak bingung memilih nilai *learning rate* yang pas.

Perlu diingat, pseudoinverse adalah operasi yang semakin berat saat jumlah data semakin banyak. Untuk melakukan pseudoinverse, semua data harus dimuat di memori terlebih dahulu. Saat data sudah sangat besar, sebaiknya kita beralih pada metode iteratif ala algoritma gradient descent dan variannya, karena algoritma tsb mampu melakukan optimasi tanpa harus memuat seluruh data di memori (i.e., dengan sistem *mini-batching*)

#### Seputar interpretasi output ELM

Seperti dijelaskan di awal, kita tidak menggunakan label asli, melainkan versi ter-*encode* label tsb. Tujuan *encode* adalah agar hasil prediksi bisa direpresentasikan sebagai vektor berisi nilai *confidence* (tingkat keyakinan) masing-masing kelas:

In [331]:
X_new = np.array([[5.2, 3.5, 4.0, 2.0]]).T
y_hat = (W2 @ sig(W1 @ X_new))

print('Output prediksi:')
print(y_hat)

Output prediksi:
[[ 0.19635205]
 [-0.18852528]
 [ 0.64047678]]


Oooops... Secara default, output ELM tidak bisa secara langsung diinterpretasikan sebagai *confidence* karena output tsb masih bukanlah distribusi probabilitas (i.e., elemen-elemen vektor output prediksi di atas jika dijumlahkan != 1), walau toh label final tetap bisa ditebak dengan mengambil indeks baris dengan nilai terbesar (indeks 2, mewakili kelas 2 (Iris versicolor)).

Kita bisa menggunakan fungsi softmax untuk memperoleh *confidence* yang (lebih) tepat:
$$softmax(\mathbf{x})=\frac{exp(\mathbf{x})}{\sum exp(\mathbf{x})}$$


In [332]:
def softmax(x):
    return np.exp(x)/np.sum(np.exp(x), axis=0)

X_new = np.array([[5.2, 3.5, 4.0, 2.0]]).T
y_hat = (W2 @ sig(W1 @ X_new))

print('Output prediksi:')
print(softmax(y_hat))

Output prediksi:
[[0.30867446]
 [0.2100635 ]
 [0.48126204]]


Artinya, data baru tersebut (X_new) merupakan jenis Iris versicolor dengan tingkat keyakinan 0.38.

Selamat mencoba.